In [1]:
import numpy as np
import astropy
import astropy.units as ur
import astropy.constants as cr
%matplotlib inline
from tdsat_rates import src_rate, bgd_rate, find_limit, compute_snr



In [2]:

td1_band = [180, 220]*ur.nm
ABmag = 23*ur.ABmag

find_limit(td1_band, diag=True)

Background Computation Integrating over Pixel Area


NameError: name 'F_λ' is not defined

In [ ]:
def find_limit(band,
               **kwargs):

    from astropy import units as ur
    
        
    exposure = kwargs.pop('exposure', 300*ur.s)
    psf_size = kwargs.pop('psf_size', 10*ur.arcsec)
    snr_limit = kwargs.pop('snr_limit', 5.0)
    diameter = kwargs.pop('diameter', 21*ur.cm)
    diag = kwargs.pop('diag', True)
    aperture_size_pixels = kwargs.pop('aperture_size_pixels', 25.)
    efficiency = kwargs.pop('efficiency', 0.87)
    qe = kwargs.pop('det_qe', 0.8)

    pixel_size = psf_size*0.5

    # Returns rates per pixel:
    nbgd_ph, nbgd_elec = bgd_rate(band=band, diag=False,diameter = diameter, pixel_size = pixel_size, **kwargs)

    # Below is now background electrons in the PSF
    bgd = nbgd_elec * aperture_size_pixels * efficiency * qe * exposure
    
    for dmag in np.arange(1000):
        magtest = (15. + dmag * 0.1) * ur.ABmag
        src_ph, src_elec = src_rate(ABmag=magtest, diag=False,diameter=diameter,band=band, **kwargs)
        
        # Turn this in to electrons per second:
        src = exposure * src_elec * efficiency * qe

        σ = np.sqrt(src + bgd)
        SNR = src / σ

        if SNR < snr_limit:

            if diag:
                print()
                print('Inputs: ')
                print('Exposure {}'.format(exposure))
                print('Optics Diameter: {}'.format(diameter))
                print('PSF Size: {}'.format(psf_size))
                print()
                print('Outputs:')
                print('Source Counts: {}'.format(src))
                print('Background Counts: {}'.format(bgd))
                print('Magnitude limit: {}'.format(magtest))
                print('Signal to noise ratio: {}'.format(SNR))
                print()
            break
    return magtest


In [ ]:
def compute_snr(band, ABmag, **kwargs):
    """
    
    
    """
    import astropy.units as ur
    exposure = kwargs.pop('exposure', 300*ur.s)
    psf_size = kwargs.pop('psf_size', 10*ur.arcsec)
    snr_limit = kwargs.pop('snr_limit', 5.0)
    diameter = kwargs.pop('diameter', 30*ur.cm)
    diag = kwargs.pop('diag', False)
    aperture_size_pixels = kwargs.pop('aperture_size_pixels', 25.)
    efficiency = kwargs.pop('efficiency', 0.87)
    qe = kwargs.pop('det_qe', 0.8)

    qe = kwargs.pop('det_qe', 0.8)

    pixel_size = psf_size*0.5

    # Returns rates per pixel:
    nbgd_ph, nbgd_elec = bgd_rate(band=band, diag=False,diameter = diameter, pixel_size = pixel_size, **kwargs)

    # Below is now background electrons in the PSF
    bgd = nbgd_elec * aperture_size_pixels * efficiency * qe * exposure
    
    src_ph, src_elec = src_rate(ABmag=ABmag, diag=False,diameter=diameter,band=band, **kwargs)
    # Turn this in to electrons per second:
    src = exposure * src_elec * efficiency * qe

    σ = np.sqrt(src + bgd)
    SNR = src / σ

    if diag:
    
        print()
        print('Exposure {}:'.format(exposure))
        print('Magnitude test: {}'.format(ABmag))
        print('Optical Efficiency: {}'.format(efficiency))
        print('Source Counts: {}'.format(src))
        print('Background Counts: {}'.format(bgd))
        print('Signal to noise ratio: {}'.format(SNR))

    return SNR



In [ ]:
# High Zodiacal background
fuv_band = [180, 220]*ur.nm
psf_size = 10*ur.arcsec
diameter=21*ur.cm
exposure=300*ur.s
maglimit = find_limit(fuv_band, snr_limit=10., exposure=exposure, diameter=diameter,
                      psf_size = psf_size, aperture_size_pixels=25, diag=True, high_zodi=True)
print(fuv_band, maglimit)
print()

nuv_band = [260, 300]*ur.nm
maglimit = find_limit(nuv_band, snr_limit=10., exposure=exposure,
                      psf_size = psf_size, aperture_size_pixels=25, diag=True, high_zodi=True)
print(nuv_band, maglimit)
print()


u_band = [340, 380]*ur.nm
maglimit = find_limit(u_band, snr_limit=10., exposure=exposure, 
                      psf_size = psf_size, aperture_size_pixels=25, diag=True, high_zodi=True)
print(u_band, maglimit)





In [ ]:
# NEP backgrounds:
# How deep do you get 10% photometry for?

fuv_band = [180, 220]*ur.nm
psf_size = 10*ur.arcsec
diameter=24*ur.cm
exposure=300*ur.s
maglimit = find_limit(fuv_band, snr_limit=10., exposure=exposure, diameter=diameter,
                      psf_size = psf_size, aperture_size_pixels=25, diag=True, high_zodi=False)
print(fuv_band, maglimit)
print()

nuv_band = [260, 300]*ur.nm
maglimit = find_limit(nuv_band, snr_limit=10., exposure=exposure,
                      psf_size = psf_size, aperture_size_pixels=25, diag=True, high_zodi=False)
print(nuv_band, maglimit)
print()


u_band = [340, 380]*ur.nm
maglimit = find_limit(u_band, snr_limit=10., exposure=exposure, 
                      psf_size = psf_size, aperture_size_pixels=25, diag=True, high_zodi=False)
print(u_band, maglimit)




In [ ]:
# Test here to see what varying the PSF does
# keeping the AB mags fixed with a 300-second exposure.


fuv_band = [180, 220]*ur.nm
nuv_band = [260, 300]*ur.nm
u_band = [340, 380]*ur.nm
diameter=24*ur.cm
exposure=900*ur.s
ABmag=22*ur.ABmag
efficiency=0.8
aperture_size_pixels=25.

psf_size = [0.1, 1, 5, 10, 12.5, 15, 20, 25]
for psf in psf_size:
    thispsf = psf*ur.arcsec
    print(thispsf)
    fuv_snr = compute_snr(fuv_band, ABmag, exposure=exposure,
                          psf_size=thispsf,efficiency = efficiency, aperture_size_pixels=aperture_size_pixels)
    nuv_snr = compute_snr(nuv_band, ABmag, exposure = exposure,
                         psf_size=thispsf, efficiency = efficiency, aperture_size_pixels=aperture_size_pixels)
    u_snr = compute_snr(u_band, ABmag, exposure=exposure,
                       psf_size=thispsf, efficiency = efficiency, aperture_size_pixels=aperture_size_pixels)

    print(fuv_snr, nuv_snr, u_snr)
    print()

In [ ]:
# Test here to see what varying the exposure time does:

fuv_band = [180, 220]*ur.nm
nuv_band = [260, 300]*ur.nm
u_band = [340, 380]*ur.nm
snr_limit = 5.
psf_size = 10*ur.arcsec
diameter=24*ur.cm

fuv_limit = []
nuv_limit = []
u_limit = []
exposure = (10**(np.arange(50)/12.5)+1)*ur.s
for exp in exposure:
    maglimit = find_limit(fuv_band, snr_limit=snr_limit, exposure=exp, diameter=diameter,
                      psf_size = psf_size, aperture_size_pixels=9, diag=False)
    fuv_limit.append(maglimit.value)
    maglimit = find_limit(nuv_band, snr_limit=snr_limit, exposure=exp, diameter=diameter,
                      psf_size = psf_size, aperture_size_pixels=9, diag=False)
    nuv_limit.append(maglimit.value)

    maglimit = find_limit(u_band, snr_limit=snr_limit, exposure=exp, diameter=diameter,
                      psf_size = psf_size, aperture_size_pixels=9, diag=False)
    u_limit.append(maglimit.value)


    




In [ ]:
plt.scatter(exposure.value, fuv_limit, label='TD1')
plt.scatter(exposure.value, nuv_limit, label = 'TD2')
plt.scatter(exposure.value, u_limit, label= 'TD3')

plt.title('Diameter {} cm, PSF {} arcsec FWHM'.format(diameter, psf_size))
plt.xscale('log')
plt.xlabel('Exposure (sec)')
plt.ylabel('Magnitude at {:2.0f}-σ'.format(snr_limit))
plt.ylim([26, 16])
plt.legend()
plt.show()

